In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from flaml import AutoML
import xgboost as xgb
from sklearn.linear_model import LinearRegression


xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

In [2]:
file_path = 'combined_dataset.nc'
combined_dataset = xr.open_dataset(file_path)

C:\Users\HP\anaconda3\envs\myenv\Lib\site-packages\ecmwflibs\__init__.py:81: UserWarning: DLL load failed while importing _ecmwflibs: 找不到指定的程序。
  warnings.warn(str(e))
C:\Users\HP\anaconda3\envs\myenv\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [3]:
df = combined_dataset.to_dataframe().reset_index()
df

,lat,lon,time,source,TREFMXAV_U,FLNS,FSNS,PRECT,PRSN,QBOT,TREFHT,UBOT,VBOT
0,49.476440,0.00,2006-01-02 00:00:00,data\003_2006_2080_0_2.nc,283.351013,60.172768,43.689503,1.322234e-08,1.171104e-17,0.005415,279.859039,4.190400,0.893759
1,49.476440,0.00,2006-01-02 00:00:00,data\003_2006_2080_352_360.nc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49.476440,0.00,2006-01-02 00:00:00,data\004_2006_2080_0_2.nc,286.786377,14.822554,18.359722,1.169902e-07,1.005727e-14,0.006941,284.214417,1.854325,8.148316
3,49.476440,0.00,2006-01-02 00:00:00,data\004_2006_2080_352_360.nc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49.476440,0.00,2006-01-02 00:00:00,data\005_2006_2080_0_2.nc,280.677002,31.676319,18.415230,3.768207e-08,1.895508e-17,0.004184,278.144806,4.251043,3.389317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149352539,58.900524,358.75,2080-12-31 00:00:00,data\031_2006_2080_352_360.nc,NaN,77.500000,11.064697,1.870194e-08,9.709327e-18,0.004028,281.875000,1.507812,-8.875000
149352540,58.900524,358.75,2080-12-31 00:00:00,data\032_2006_2080_0_2.nc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149352541,58.900524,358.75,2080-12-31 00:00:00,data\032_2006_2080_352_360.nc,NaN,59.022911,8.976113,5.294515e-08,8.997415e-17,0.004286,281.492950,3.020460,-0.110520
149352542,58.900524,358.75,2080-12-31 00:00:00,data\033_2006_2080_0_2.nc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
clean_df = df.dropna()
clean_df

,lat,lon,time,source,TREFMXAV_U,FLNS,FSNS,PRECT,PRSN,QBOT,TREFHT,UBOT,VBOT
0,49.476440,0.00,2006-01-02 00:00:00,data\003_2006_2080_0_2.nc,283.351013,60.172768,43.689503,1.322234e-08,1.171104e-17,0.005415,279.859039,4.190400,0.893759
2,49.476440,0.00,2006-01-02 00:00:00,data\004_2006_2080_0_2.nc,286.786377,14.822554,18.359722,1.169902e-07,1.005727e-14,0.006941,284.214417,1.854325,8.148316
4,49.476440,0.00,2006-01-02 00:00:00,data\005_2006_2080_0_2.nc,280.677002,31.676319,18.415230,3.768207e-08,1.895508e-17,0.004184,278.144806,4.251043,3.389317
6,49.476440,0.00,2006-01-02 00:00:00,data\006_2006_2080_0_2.nc,280.960754,88.106346,51.294487,7.633440e-11,0.000000e+00,0.003503,275.069061,-1.688833,2.870790
8,49.476440,0.00,2006-01-02 00:00:00,data\007_2006_2080_0_2.nc,283.172455,68.380234,32.079136,2.302120e-08,9.563444e-18,0.004261,281.727570,11.090128,-2.101207
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145958159,58.900524,356.25,2080-12-31 00:00:00,data\029_2006_2080_352_360.nc,284.386536,65.868126,7.963246,1.040656e-07,4.552225e-13,0.004434,282.599701,15.513151,7.881047
145958161,58.900524,356.25,2080-12-31 00:00:00,data\030_2006_2080_352_360.nc,284.329071,55.820065,6.880709,5.798378e-08,4.155667e-17,0.004786,282.764069,7.697163,4.050290
145958163,58.900524,356.25,2080-12-31 00:00:00,data\031_2006_2080_352_360.nc,283.315796,75.500000,12.151611,2.201932e-08,8.859965e-17,0.004242,282.000000,2.984375,-8.437500
145958165,58.900524,356.25,2080-12-31 00:00:00,data\032_2006_2080_352_360.nc,283.151489,50.954922,5.851292,9.209941e-08,3.635996e-16,0.004868,281.111633,1.075335,1.956308


In [5]:
df = clean_df

In [6]:
#deal with time information
df['time'] = df['time'].astype(str)
df['time'] = pd.to_datetime(df['time'])

C:\Users\HP\AppData\Local\Temp\ipykernel_5144\3966807225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = df['time'].astype(str)
C:\Users\HP\AppData\Local\Temp\ipykernel_5144\3966807225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


In [7]:
#Split time to year, month, day of year
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['dayofyear'] = df['time'].dt.dayofyear


C:\Users\HP\AppData\Local\Temp\ipykernel_5144\3916499746.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['time'].dt.year
C:\Users\HP\AppData\Local\Temp\ipykernel_5144\3916499746.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['time'].dt.month
C:\Users\HP\AppData\Local\Temp\ipykernel_5144\3916499746.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [8]:
df['WindSpeed'] = np.sqrt(df['UBOT']**2 + df['VBOT']**2)

C:\Users\HP\AppData\Local\Temp\ipykernel_5144\890795316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['WindSpeed'] = np.sqrt(df['UBOT']**2 + df['VBOT']**2)


In [9]:
#water_vapor_content
P = 101325 
R = 287.05
df['WaterVapor'] =  (df['QBOT'] * P) / ( R * df['TREFHT'])

C:\Users\HP\AppData\Local\Temp\ipykernel_5144\822635245.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['WaterVapor'] =  (df['QBOT'] * P) / ( R * df['TREFHT'])


In [10]:
#heat_flux
rho = 1.225 
Cp = 1005 
df['HeatFlux'] = rho * Cp * df['TREFHT'] * df['WindSpeed']

C:\Users\HP\AppData\Local\Temp\ipykernel_5144\2904721142.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HeatFlux'] = rho * Cp * df['TREFHT'] * df['WindSpeed']


cols = ['TREFMXAV_U'] + [col for col in df.columns if col != 'TREFMXAV_U']
correlation_matrix = df[cols]

correlation_matrix = correlation_matrix.drop(['source','time'], axis=1).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Variables')
plt.show()

In [11]:
#df = df.drop(['UBOT','VBOT'],axis=1)

In [12]:
#split train and test
train = df[df['year'] < 2040]
valid = df[(df['year'] >= 2040) & (df['year'] < 2050)]
test = df[df['year'] >= 2050]
#test = test[test['time'].dt.year < 2030]

In [13]:
#train and X_test
X_train = train.drop(['TREFMXAV_U','source','time'], axis=1) 
y_train = train['TREFMXAV_U']
X_valid = valid.drop(['TREFMXAV_U','source','time'], axis=1) 
y_valid = valid['TREFMXAV_U']
X_test = test.drop(['TREFMXAV_U','source','time'], axis=1) 
y_test = test['TREFMXAV_U']

In [ ]:
############################

dtrain = xgb.DMatrix(X_train, label=y_train)


param = {
    'max_depth': 6,
    'min_child_weight': 1,
    'eta': .3,
    'subsample': 1,
    'colsample_bytree': 1,
    'objective': 'reg:squarederror',
}


gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(3, 10)
    for min_child_weight in range(1, 6)
]


min_rmse = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(max_depth, min_child_weight))
    
    param['max_depth'] = max_depth
    param['min_child_weight'] = min_child_weight
    
    cv_results = xgb.cv(
        param,
        dtrain,
        num_boost_round=100,
        seed=42,
        nfold=3,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth, min_child_weight)

print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))


In [14]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

params = {
    'max_depth': 9, 
    'min_child_weight':4,
    'eta': 0.3,     
    'objective': 'reg:squarederror', 
    'nthread': 4    
}
#############
num_max_rounds = 5000  

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_max_rounds,
    evals=[(dvalid, "Validation")],
    early_stopping_rounds=10
)


print("Best iteration:", model.best_iteration + 1)


[0]	Validation-rmse:3.74234
[1]	Validation-rmse:2.73065
[2]	Validation-rmse:2.05315
[3]	Validation-rmse:1.61292
[4]	Validation-rmse:1.32937
[5]	Validation-rmse:1.15622
[6]	Validation-rmse:1.03928
[7]	Validation-rmse:0.97590
[8]	Validation-rmse:0.92184
[9]	Validation-rmse:0.89464
[10]	Validation-rmse:0.86818
[11]	Validation-rmse:0.85462
[12]	Validation-rmse:0.83781
[13]	Validation-rmse:0.82448
[14]	Validation-rmse:0.81698
[15]	Validation-rmse:0.80948
[16]	Validation-rmse:0.79836
[17]	Validation-rmse:0.79355
[18]	Validation-rmse:0.78088
[19]	Validation-rmse:0.77673
[20]	Validation-rmse:0.77297
[21]	Validation-rmse:0.77057
[22]	Validation-rmse:0.76421
[23]	Validation-rmse:0.76183
[24]	Validation-rmse:0.75513
[25]	Validation-rmse:0.75307
[26]	Validation-rmse:0.75064
[27]	Validation-rmse:0.74683
[28]	Validation-rmse:0.74528
[29]	Validation-rmse:0.74228
[30]	Validation-rmse:0.74036
[31]	Validation-rmse:0.73680
[32]	Validation-rmse:0.73567
[33]	Validation-rmse:0.73413
[34]	Validation-rmse:0.7

In [15]:
model.save_model('xgb_model.json')
model.save_model('xgb_model.bin')

C:\Users\HP\anaconda3\envs\myenv\Lib\site-packages\xgboost\core.py:160: UserWarning: [08:55:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [16]:
#X_test = X_test.drop(['predict'],axis=1)
y_pred = model.predict(dtest)
#test_mean['predict'] = y_pred
test['predict'] = y_pred

C:\Users\HP\AppData\Local\Temp\ipykernel_5144\2080719661.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predict'] = y_pred


In [17]:
#y_test = test['TREFMXAV_U']
y_test_pred = test['predict']

In [18]:
#mse = mean_squared_error(y_test_mean, y_test_pred)
#mae = mean_absolute_error(y_test_mean, y_test_pred)
#r2 = r2_score(y_test_mean, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)
rmse = sqrt(mse)



print(f'Validation Mean Squared Error (MSE): {mse}')
print(f'Validation Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Validation R-squared: {r2}')

Validation Mean Squared Error (MSE): 0.5135531425476074
Validation Mean Absolute Error (MAE): 0.5262275338172913
Root Mean Squared Error (RMSE): 0.7166262223416106
Validation R-squared: 0.9824598186772603
